In [2]:
# Import necessary packages
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Load the data set that contains distances between 
# cardiac arrests and potential AED locations
covered = pd.read_csv('Example2_distanceMatrix.csv', index_col='ID')

# PARAMETERS
# Coverage cutoff limit 
coverage_distance = 100

# Number of AEDs to be placed 
K = 25

# Compute a_ij, which states whether each cardiac arrest is
# within the coverage distance of each potential AED location
covered = (covered <= coverage_distance).astype(int)
n_cases = covered.shape[0]
n_candidates = covered.shape[1]
A = pd.DataFrame.to_numpy(covered)

# Create a Gurobi model object
m = gp.Model('AED')

# DECISION VARIABLES
# x is whether an AED is placed at potential AED location j
x = m.addMVar(n_candidates, vtype = GRB.BINARY, name="x")

# y is whether cardiac arrest i is covered by at least one AED
y = m.addMVar(n_cases, vtype = GRB.BINARY, name="y")

# CONSTRAINTS
# Cardiac arrest covered if at least one AED is 
# placed within range
m.addConstr(A@x >= y)

# Total number of AEDs placed
m.addConstr(x.sum() == K)

# OBJECTIVE
# Maximize the number of covered cardiac arrests
m.setObjective(y.sum(), gp.GRB.MAXIMIZE)

# Run the optimization model
m.optimize()

# Print the optimal solution and its objective value
selected = []
for j in x.tolist():
    if j.X > 0.5: 
        selected.append(j.index)
print("Selected locations: ", end="")
print(*selected, sep=', ')
print("Number of covered cardiac arrests: " + str(int(m.objVal)))

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 501 rows, 2000 columns and 2454 nonzeros
Model fingerprint: 0x00a16fe7
Variable types: 0 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 463 rows and 1909 columns
Presolve time: 0.00s
Presolved: 38 rows, 91 columns, 160 nonzeros
Found heuristic solution: objective 145.0000000
Variable types: 0 continuous, 91 integer (83 binary)

Root relaxation: objective 1.550000e+02, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     155.0000000  155